# Airline Sample Data Exercise Answer

### **Q1:** How many unique customers are there? How many do not have specific age information?

In [1]:
import smv

In [5]:
# Input data
# openCsv:  Helper functions available in SMV's Python shell
customer = openCsv(r'hdfs:///user/ml/data/input/airline/customer.csv')
member = openCsv(r'hdfs:///user/ml/data/input/airline/member_lvl.csv')
bkg = openCsv(r'hdfs:///user/ml/data/input/airline/booking.csv')
tkt = openCsv(r'hdfs:///user/ml/data/input/airline/ticket.csv')
ckin = openCsv(r'hdfs:///user/ml/data/input/airline/checkin.csv')
geo = openCsv(r'hdfs:///user/ml/data/input/airline/ref_geo.csv')

In [6]:
type(customer)

pyspark.sql.dataframe.DataFrame

In [7]:
# smvEdd: This module provides the helper functions on DataFrame objects and Column objects
customer.smvEdd()

CUST_ID              Non-Null Count         22
CUST_ID              Null Count             0
CUST_ID              Min Length             12
CUST_ID              Max Length             12
CUST_ID              Approx Distinct Count  22
BIRTH_MM             Non-Null Count         19
BIRTH_MM             Null Count             3
BIRTH_MM             Average                7.0
BIRTH_MM             Standard Deviation     2.943920288775949
BIRTH_MM             Min                    2.0
BIRTH_MM             Max                    12.0
BIRTH_DD             Non-Null Count         19
BIRTH_DD             Null Count             3
BIRTH_DD             Average                16.789473684210527
BIRTH_DD             Standard Deviation     8.495612902933562
BIRTH_DD             Min                    1.0
BIRTH_DD             Max                    30.0
BIRTH_YYYY           Non-Null Count         19
BIRTH_YYYY           Null Count             3
BIRTH_YYYY           Average                1974.157894736

### **A1:** 22 unique customers, 3 without age information.

### =========================================================================================

### **Q2:** Which customer joined FFP the earliest among all sampled customers?

In [8]:
member.show()

+------------+--------+----------+
|     CUST_ID|CURR_LVL| ENROLL_DT|
+------------+--------+----------+
|000365709222|       5|1982-09-15|
|000171572471|       0|1995-12-06|
|000134299922|       0|      null|
|000407811114|       0|      null|
|000025063186|       1|      null|
|000102195898|       0|2007-02-17|
|000809758842|       0|2004-06-21|
|000493524149|       0|      null|
|000099281768|       0|2010-12-29|
|000876964176|       1|      null|
|000704225906|       0|2001-03-24|
|000499804303|       3|2002-03-05|
|000531414908|       0|2000-06-11|
|000950169665|       2|      null|
|000364119910|       0|1986-05-15|
|000682324216|       0|2007-10-21|
|000165855849|       0|1995-06-19|
|000990013836|       0|2010-02-14|
|000225479520|       0|2014-03-22|
|000544814415|       0|2015-04-05|
+------------+--------+----------+
only showing top 20 rows



In [4]:
member.filter(col("ENROLL_DT").isNotNull()).orderBy("ENROLL_DT").show(1)

+------------+--------+----------+
|     CUST_ID|CURR_LVL| ENROLL_DT|
+------------+--------+----------+
|000365709222|       5|1982-09-15|
+------------+--------+----------+
only showing top 1 row



### **A2:** The customer with ID 000365709222 joined in 1982.

### =================================================================================

### **Q3:** How many records are there in ticket table?

In [5]:
tkt.count()

1941

### **A3:** 1941 records

One caveat is that this kind of check may take a long time on a large full data.

### ==================================================================================

### **Q4:** What is the missing rate of "CABIN" in booking table?

In [9]:
bkg.show()

+------+---------------+------------+--------------------+--------------------+-------+----------+---------------+-------------------+-------------------+---------------+---------------+-----+-----+-------+---------+-------+---------+---------+
|   PNR|PNR_CREATION_DT|     CUST_ID|        BK_ACTIVE_TM|      BK_INACTIVE_TM|FLT_NUM|CARRIER_CD|FLT_LEG_DPRT_DT|    FLT_LEG_DPRT_TM|    FLT_LEG_ARRV_TM|SCH_LEG_ORIG_CD|SCH_LEG_DEST_CD|CABIN|CLASS|NUM_PAX|ACTION_CD|SEG_NUM|CONN_NEXT|CONN_PREV|
+------+---------------+------------+--------------------+--------------------+-------+----------+---------------+-------------------+-------------------+---------------+---------------+-----+-----+-------+---------+-------+---------+---------+
|00E41B|     2014-03-04|000365709222|2014-03-04 01:59:...|2014-03-04 01:59:...|   8113|        FB|     2014-03-12|2014-03-13 00:16:00|2014-03-13 01:24:00|            MBS|            ORD|    Y|    Y|      1|       TV|      1|        2|        0|
|00E41B|     2014-03

In [10]:
bkg.count()

6676

In [6]:
bkg_sample = bkg.sample(False, 0.2, 99).cache()
bkg_sample.count()

1309

In [7]:
bkg_sample.smvEdd("CABIN")

CABIN                Non-Null Count         1286
CABIN                Min Length             1
CABIN                Max Length             1
CABIN                Approx Distinct Count  4


In [8]:
float(1309-1286)/float(1309)*100

1.757066462948816

### **A4:** 1.76%

### ======================================================================================

### **Q5:** Validate whether the set of ["CUST_ID", "PNR", "PNR_CREATION_DT", "FLT_LEG_DPRT_DT", "SCH_LEG_ORIG_CD", "SCH_LEG_DEST_CD"] can be primary keys of ticket table.

In [9]:
tkt_sample = tkt.sample(False, 0.3, 99).cache()
tkt_sample.count()

583

In [10]:
#Remove duplicate records from the DataFrame by arbitrarily selecting the first record from a set of records with same primary key or 
#key combo.
tkt_sample.smvDedupByKey("CUST_ID", "PNR", "PNR_CREATION_DT", "FLT_LEG_DPRT_DT", "SCH_LEG_ORIG_CD", "SCH_LEG_DEST_CD").count()

583

### **A5:** Yes

### ==========================================================================================

### **Q6:** What was the most frequent check-in source among sampled customers?

In [11]:
ckin.columns

['FLT_LEG_DPRT_DT',
 'FLT_NUM',
 'CKIN_CARRIER_CD',
 'SCH_LEG_ORIG_CD',
 'CKIN_SEQ_NUM',
 'CKIN_PAX_NUM',
 'SEAT',
 'CKIN_TM',
 'CKIN_CLASS',
 'CKIN_SOURCE',
 'CUST_ID',
 'PNR',
 'PNR_CREATION_DT']

In [12]:
ckin.groupBy('CKIN_SOURCE').count().collect()

[Row(CKIN_SOURCE=u'AGENT', count=586),
 Row(CKIN_SOURCE=u'WEB', count=329),
 Row(CKIN_SOURCE=u'eService', count=155)]

In [11]:
ckin.smvFreqHist("CKIN_SOURCE")

Histogram of CKIN_SOURCE: String sorted by Frequency
key                      count      Pct    cumCount   cumPct
AGENT                      586   54.77%         586   54.77%
WEB                        329   30.75%         915   85.51%
eService                   155   14.49%        1070  100.00%
-------------------------------------------------


### **A6:** "AGENT" - typically means a staff checked a passenger in at the airport counter

### ====================================================================================

### **Q7:** At which city and which country does "AGP" locate?

In [12]:
geo.filter(col("LOCATION_ID")=="AGP").select("CITY_DESC", "COUNTRY_DESC").show()

+---------+------------+
|CITY_DESC|COUNTRY_DESC|
+---------+------------+
|   Malaga|       Spain|
+---------+------------+



### **A7:** Malaga, Spain